In [22]:
import pandas as pd 
import numpy as np 
import json
import os
import statsmodels.api as sm

import warnings 
warnings.filterwarnings(action='ignore') 

2차 전처리  (여기부터 Streamlit에 추가하기) 
* 1. activity_merged, contactDB 합치기  (행동데이터 기준으로) 
    * 여정, Y1, Y2, duration, quantile,  
    * 행동변수 빈도 수 count
    * Offering 변수 생성 
    * 이항형/연속형 버전 저장 
* 2. 모델링 용 데이터 만들기 (빈도/범주)

In [23]:
import os 
activity_path = '../../data/processed1/'

os.listdir(activity_path)

['ContactDB.csv', 'activity_merged.csv', 'EmailSent.csv']

In [24]:

def get_dates(contactid, idx_list,ordered_df): # 고객 별 첫 행동과 마지막 행동의 시간을 가져오는 함수
    first_date = []; last_date = []
    df = ordered_df[ordered_df.ContactID == contactid].reset_index(drop=True)
    if idx_list[0] == '육성':
        first = 0; last = -1
        first_date.append(df.iloc[first,2])
        last_date.append(df.iloc[last,2])
    else:
        first1 = 0; last1 = idx_list[0]-1
        first_date.append(df.iloc[first1,2])
        last_date.append(df.iloc[last1,2])
    return first_date, last_date 

#2. 파생변수 생성 (Y2, 여정): 리드생성 여부 파악 (counsel/inquiry) & 고객 별 여정 list 
def get_journey_info(temp,ordered_df): # 한 고객에 대한 첫 행동 날짜, 마지막 행동 날짜, 여정, 리드생성 여부(Y2) 를 가진 데이터프레임  생성 
# printing original list
    L = temp.ActivityType
    contactid = temp['ContactID']
    contactid_list = [];first_date = []; journey_list = []; y2 = []
    
    if ('counsel' in L) | ('inquiry' in L):
        size = len(L)
        idx_list = [idx + 1 for idx, val in
                    enumerate(L) if (val == 'counsel') or (val == 'inquiry')]
        journey_list = [L[i: j] for i, j in zip([0] + idx_list, idx_list +([size] if idx_list[-1] != size else []))]
        journey_list = [journey_list[0]]
        y2.append(1)
        
    else:
        journey_list = [L]
        idx_list = ['육성']
        y2.append(0)
        
    contactid_list = [contactid]*1
    first_date, last_date = get_dates(contactid, idx_list,ordered_df) # get_dates 함수 사용
    if '.' in first_date:
        first_date = first_date.split('.')[0]
    else:
        first_date = first_date
    if '.' in last_date:
        last_date = last_date.split('.')[0]
    
    data = pd.DataFrame({'ContactID': contactid_list, '첫행동날짜':first_date,
                        '리드생성/마지막날짜':last_date,'여정':journey_list, 'Y2':y2})
    return data

In [25]:
def get_initial_dataframe(activity_path):
    # Load Dataset 
    activities = pd.read_csv(activity_path + 'activity_merged.csv', low_memory = False, usecols = ['ContactID', 'ActivityType', 'ActivityDate','Offering'])
    # 고객 단위로 여정 리스트 생성 
    # 고객 ID, 활동시간 순으로 정렬
    ordered_df = activities.sort_values(['ContactID', 'ActivityDate'])
    ordered_df.reset_index()
    # 고객 별 모든 행동이력이 포함된 리스트 생성 
    temp = ordered_df.groupby(['ContactID'])['ActivityType'].aggregate(lambda x: x.tolist()).reset_index()
    #  함수를 사용하여 고객 별 여정, 첫 행동과 마지막 행동의 날짜, 리드생성 여부가 포함된 dataframe을 생성한다. 
    dataframe = []
    for i in range(len(temp)): # 모든 고객에 대한 함수 실행 후 concat 
        df = temp.iloc[i,:]
        dataframe.append(get_journey_info(df, ordered_df))
    data = pd.concat(dataframe)
    return data, ordered_df

In [26]:
# 파생변수 생성 (duration): 첫 행동과 마지막 행동에 대한 시간 정보 이용
# 계산된 행동 간의 시간을 정수로 바꾼다
def date_as_float(date):
    if date != '0':
        return round(pd.Timedelta(date)/ pd.to_timedelta(1,unit= 'D') ,2)
    else:
        return float(0)
    
# 문자열을 시간으로 type 변환 
def get_new_columns(data):
    # 파생변수 생성: duration 
    data['첫행동날짜'] = data['첫행동날짜'].apply(lambda x: x.split('.')[0])
    data['리드생성/마지막날짜'] = data['리드생성/마지막날짜'].apply(lambda x: x.split('.')[0])
    first_action_date = pd.to_datetime(data['첫행동날짜'], format = '%Y-%m-%d %H:%M:%S')
    last_date = pd.to_datetime(data['리드생성/마지막날짜'], format = '%Y-%m-%d %H:%M:%S')
    data['y3'] = last_date - first_action_date
    data['duration'] = data.y3.apply(date_as_float) # date_as_float 함수 사용
    
    # 파생변수 생성: Y1 
    data['Y1'] = data['duration'].apply(lambda x: 0 if x < 1 else 1)
    data.reset_index(inplace=True, drop=True)
    
    # 파생변수 생성: 고객의 여정 4분위수 기준으로 라벨링
    data['quantile'] = pd.qcut(data[data.Y1 == 1]['duration'], 4, labels=False)
    return data

In [27]:
# 각 고객, 여정 별 행동의 빈도 세기 
def get_counts(journey):
    temp_df = pd.DataFrame(pd.Series(journey).value_counts())
    df = np.transpose(temp_df)
    return df

def get_counts_df(data): # 여정 리스트 안의 행동유형의 빈도 세는 함수
    temp = []
    for i in range(len(data)):
        temp.append(get_counts(list(data.여정)[i]))  # get_counts 함수 사용
    counts = pd.concat(temp)
    counts.fillna(0,inplace = True)
    counts.reset_index(drop=True,inplace=True)
    data_final = pd.concat([data, counts], axis = 1)
    use_cols = ['ContactID', '첫행동날짜', '리드생성/마지막날짜', '여정','Y1', 'Y2', 'duration','quantile', 
                'webvisit', 'EmailClickthrough', 'EmailOpen', 'register','subscribe','download',
                'counsel', 'survey', 'event', 'inquiry', 'service_request', 'attend']
    data_final = data_final[use_cols]
    data_final = data_final.fillna(0)
    return data_final

def get_binary_counts(data): # 행동변수 이항형으로 바꾸는 함수
    data2 = data.copy()
    activity_list = ['webvisit', 'EmailClickthrough', 'EmailOpen', 'register',
       'subscribe', 'download', 'counsel', 'survey', 'event',
       'inquiry', 'service_request', 'attend']
    for activity in activity_list:
        data2[activity] = data2[activity].apply(lambda x: 1 if x>0 else 0)
    return data2

In [28]:
activity_path = '../../data/processed1/'

data, ordered_df = get_initial_dataframe(activity_path)
data =  get_new_columns(data)
data_final = get_counts_df(data)#행동변수 연속형 
data_final_binary =  get_binary_counts(data_final) # 행동변수 이항형 

# 오퍼링 관심도 

In [44]:
# 가중치 설정 (더미변수 기준)

def min_max_scale(coeff_list): # min_max scaling
    minimum = min(coeff_list); maximum = max(coeff_list)
    numerator = np.subtract(coeff_list+1, min(coeff_list)) #최소값이 0이 되지 않게 하기 위해 1을 더해줌. 
    denominator = maximum - minimum 
    scaled_list = (numerator / denominator )
    scaled_list = scaled_list
    ratio_list = scaled_list / np.sum(scaled_list)
    return np.round(ratio_list,2)


def get_activity_weight(data_final_binary): #회귀분석 & 유의한 변수만 사용해 가중치 설정
    # 범주형 행동변수에 대한 회귀분석 시행 
    X = data_final_binary[['quantile', 'webvisit', 'EmailClickthrough', 'EmailOpen', 'register',
           'subscribe', 'download','survey', 'event', 'service_request', 'attend']]
    y = data_final_binary.Y2
    X_add = sm.add_constant(X)
    model = sm.Logit(y.values.reshape(-1,1),X_add)
    results = model.fit()
    summary = results.summary()

    df = pd.read_html(summary.tables[1].as_html())[0].iloc[2:,[0,1,4]].reset_index(drop=True)
    df.columns = ['variable', 'coef', 'p_val']
    to_use = df[df.p_val.astype('float') <0.05] #p-value 가 0.05 이하인 변수에 대해서만 가중치 설정
    variable = to_use.variable[1:] 
    coef = to_use.coef[1:].astype('float')
    weight = min_max_scale(coef)
    df = pd.DataFrame({'ActivityType':variable, 'weight':weight})
    return df

# 고객의 오퍼링 관심도 scoring 
def get_interest_rank(activity_path, weight_df):
    # 고객의 오퍼링 별 행동빈도 구하기 
    activity = pd.read_csv(activity_path + 'activity_merged.csv', low_memory = False, usecols = ['ContactID', 'ActivityType', 'ActivityDate','Offering'])
    #activity = activity[activity.Offering.notna()]
   
#     colname = [] # 구분을 위해 변수명 지정
#     for i in range(len(counts.columns)):
#         s = list(counts.columns)[i][0]+'_'+list(counts.columns)[i][1]
#         colname.append(s)
#     counts.columns = colname

    interest_score = pd.pivot_table(activity[['ActivityType', 'ActivityDate','ContactID', 'Offering']],                # 피벗할 데이터프레임
                         index = ['ContactID', 'Offering'],    # 행 위치에 들어갈 열
                         columns = 'ActivityType',    # 열 위치에 들어갈 열
                         aggfunc = 'count')   
    interest_score  = interest_score.reset_index()
    interest_score.columns = ['ContactID', 'Offering','EmailClickthrough', 'EmailOpen', 'attend', 'counsel',
                   'download','event', 'inquiry', 'register','service_request', 'subscribe', 'survey', 'webvisit']
    interest_score.fillna(0, inplace = True)
    interest_score = interest_score[interest_score.Offering != '0']
    interest_score  = interest_score .drop(['counsel', 'inquiry'],axis=1)

    for i, j in zip(weight_df.ActivityType, weight_df.weight):
        interest_score.loc[:, i] = interest_score.loc[:, i]*j

    interest_score['interest_score'] = interest_score[weight_df.ActivityType].sum(axis = 1)
    interest_score['offer_rank'] = interest_score.groupby('ContactID')['interest_score'].rank(method = 'first',ascending = False)
    interest_score = interest_score[['ContactID', 'Offering','interest_score', 'offer_rank']].sort_values(['ContactID','offer_rank'])
    interest_score.reset_index(drop=True, inplace=True)
    interest_score.head()

    rank = interest_score[['ContactID', 'Offering','offer_rank']]
    rank = rank.pivot(index = 'ContactID', columns = 'offer_rank')
    ranks = rank.iloc[:, 0:3]
    ranks.columns = ['고객관심rank1', '고객관심rank2', '고객관심rank3']
    ranks.reset_index(drop=False, inplace=True)
    return ranks

In [38]:
activity_path = '../../data/processed1/'
activity = pd.read_csv(activity_path + 'activity_merged.csv', low_memory = False, usecols = ['ContactID', 'ActivityType', 'ActivityDate','Offering'])
#activity = activity[activity.Offering.notna()]

#     colname = [] # 구분을 위해 변수명 지정
#     for i in range(len(counts.columns)):
#         s = list(counts.columns)[i][0]+'_'+list(counts.columns)[i][1]
#         colname.append(s)
#     counts.columns = colname

interest_score = pd.pivot_table(activity[['ActivityType', 'ActivityDate','ContactID', 'Offering']],                # 피벗할 데이터프레임
                     index = ['ContactID', 'Offering'],    # 행 위치에 들어갈 열
                     columns = 'ActivityType',    # 열 위치에 들어갈 열
                     aggfunc = 'count')   
interest_score  = interest_score.reset_index()

interest_score.columns = ['ContactID', 'Offering','EmailClickthrough', 'EmailOpen', 'attend', 'counsel',
               'download','event', 'inquiry', 'register','service_request', 'subscribe', 'survey', 'webvisit']


In [45]:
def get_email_sent_offering(activity_path, ranks):
    emailsent = pd.read_csv(activity_path + 'EmailSent.csv')
    emailsent.rename(columns = {'ContactId':'ContactID'},inplace=True)
    send = emailsent.groupby(['ContactID','Offering'])['ContactID'].count().unstack()
    send.reset_index(inplace=True)
    send.fillna(0, inplace = True)
    #send.drop(['TEST', 'unknown'], axis = 1, inplace=True)
    send_count = pd.melt(send, id_vars = 'ContactID', value_vars = ['aidata', 'cloud', 'consult', 'dcx', 'dxsol', 'fin',
           'general', 'infra', 'secu'])
    send_count = send_count.sort_values(['ContactID', 'value'])
    send_count.reset_index(drop=True, inplace=True)

    send_count['sent_offer'] = send_count.groupby('ContactID')['value'].rank(method = 'first',ascending = False)
    send_count2 = send_count[['ContactID', 'Offering', 'sent_offer']]

    send_rank = send_count2.pivot(index = 'ContactID', columns = 'sent_offer')
    send_rank.reset_index(inplace=True)
    send_rank = send_rank.iloc[:, 0:2]
    send_rank.columns = ['ContactID','sent_offer']

    merged = pd.merge(send_rank, ranks, on='ContactID', how = 'right')
    return merged

In [46]:
def get_match_col(merged):
    match1 = merged['sent_offer'] == merged['고객관심rank1']
    match1 = match1.astype(int)
    match2 = merged['sent_offer'] == merged['고객관심rank2']
    match2 = match2.astype(int)
    match3 = merged['sent_offer'] == merged['고객관심rank3']
    match3 = match3.astype(int)

    merged['match1'] = match1
    merged['match2'] = match2
    merged['match3'] = match3
    
    merged['match'] = merged[['match1', 'match2', 'match3']].sum(axis = 1)
    merged['match'] = (merged['match']>0).astype(int)
    use_cols = ['ContactID', 'sent_offer', '고객관심rank1', '고객관심rank2', '고객관심rank3', 'match']
    return merged[use_cols]

In [47]:
activity_path = '../../data/processed1/'

weight_df = get_activity_weight(data_final_binary)
ranks = get_interest_rank(activity_path, weight_df)
merged = get_email_sent_offering(activity_path, ranks)
offering = get_match_col(merged)

Optimization terminated successfully.
         Current function value: 0.107621
         Iterations 10


In [48]:
def get_continuous_df(contact_path, data_final, offering, save):
    contact = pd.read_csv(contact_path + 'ContactDB.csv')
    merged = pd.merge(data_final, offering, on = 'ContactID', how = 'left')
    data = pd.merge(contact, merged, on = 'ContactID', how = 'right')
    if save == 'y':
        data.to_csv('../../data/processed2/data_연속형행동.csv',index=False)
        print('Data successfully saved')
    return data 

def get_binary_df(contact_path, data_final_binary, offering, save):
    contact = pd.read_csv(contact_path + 'ContactDB.csv')
    merged = pd.merge(data_final_binary, offering, on = 'ContactID', how = 'left')
    data = pd.merge(contact, merged, on = 'ContactID', how = 'right')
    if save == 'y':
        data.to_csv('../../data/processed2/data_이항형행동.csv',index=False)
        print('Data successfully saved')
    return data 


In [49]:
contact_path = '../../data/processed1/'
offering_continuous = get_continuous_df(contact_path,data_final, offering, 'y')
offering_binary = get_binary_df(contact_path, data_final_binary, offering, 'y')

Data successfully saved
Data successfully saved


In [51]:
def get_dummy(df,columns,base_value=None):
    '''
    df : 데이터프레임
    columns : 가변수로 변환할 칼럼들
    base_value : {가변수로 변환할 칼럼명 : 제외시킬 범주}
    '''
    for c in columns:
        num_level = len(set(df[c])) ## 유니크한 원소의 개수
        df[c] = df[c].astype('str')
        uniq_element = sorted(list(set(df[c])))[1:] ## 정렬했을 때 첫 번째 범주를 제외
        
        if base_value: ## 제외시킬 범주가 있을 경우 해당 범주를 제외
            if c in base_value.keys():
                assert base_value[c] in list(set(df[c])), f'{base_value[c]} is not contained in {c}'
                uniq_element = sorted(list(set(df[c])-{base_value[c]}))
                    
        data = dict()
        for i in range(num_level-1):
            dummy_data = []
            val = uniq_element[i]
            for d in df[c]: ## 해당 범주인 경우만 1 나머지는 0
                if d == val:
                    dummy_data.append(1)
                else:
                    dummy_data.append(0)
            col_name = c+'_'+str(val) ## 갸변수의 칼럼명 지정
            data[col_name] = dummy_data

        temp_df = pd.DataFrame(data)
        df = pd.concat([df.reset_index(drop=True),temp_df],axis=1) ## 가변수 칼럼을 원 데이터 뒤에 결합
        ## reset_index를 이용하여 로우 인덱스를 초기화해야함.

    return df.drop(columns=columns) ## 원 범주 칼럼은 제외


def data_for_modelling(data2):
    data = data2.copy()
    data['first_action'] = data['first_action'].apply(lambda x: 1 if x == 'ib' else 0) #inbound면 1
     # 특성변수 범주화 
    data.Title = data.Title.fillna('unknown')
    data.Industry = data.Industry.fillna('산업_unknown')
    data.isfam = data.isfam.fillna(0)
    data.first_action = data.first_action.fillna(0)
    #data['first_action'] = data['first_action'].apply(lambda x: '1' if x == 'ib' else '0') #inbound면 1
    data.고객등급 = data.고객등급.fillna('unknown')
   
    column_list = ['Title', 'Industry', 'isfam', 'first_action', '고객등급']
    # 가변수화 할 범주 
    base_val = {'Title': 'unknown', 'Industry':'산업_unknown', 
                'isfam':'0.0','first_action':'0','고객등급':'unknown'}
    
    to_categorize = data.dropna(subset = ['Title','Industry','isfam','first_action','고객등급'], axis = 0)
    only_activity = data.drop(to_categorize.index)
    data  = get_dummy(data,column_list, base_val)
    

    return data
    
    

In [52]:
continuous = data_for_modelling(offering_continuous)
binary = data_for_modelling(offering_binary)

In [53]:
continuous.describe()

,ContactID,CompanyID,Mapping,Y1,Y2,duration,quantile,webvisit,EmailClickthrough,EmailOpen,...,Industry_산업_제조,isfam_1.0,first_action_1,고객등급_1,고객등급_2,고객등급_3,고객등급_4,고객등급_5,고객등급_6,고객등급_7
count,28697.000000,11594.000000,22017.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,...,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000
mean,19573.246158,1492.058565,0.526593,0.736035,0.055999,168.148122,1.103878,4.913963,1.906053,15.481653,...,0.116911,0.086560,0.163710,0.118096,0.007597,0.024741,0.013660,0.168136,0.020908,0.050876
std,11550.173277,364.523211,0.499304,0.440788,0.229924,175.153949,1.164946,16.270271,8.058809,49.919611,...,0.321320,0.281193,0.370019,0.322727,0.086828,0.155338,0.116077,0.373994,0.143079,0.219749
min,1.000000,899.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8490.000000,1308.000000,0.000000,0.000000,0.000000,0.120000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,20582.000000,1457.000000,1.000000,1.000000,0.000000,105.910000,1.000000,1.000000,0.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,29323.000000,1645.000000,1.000000,1.000000,0.000000,325.170000,2.000000,4.000000,1.000000,19.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,38999.000000,3493.000000,1.000000,1.000000,1.000000,614.060000,3.000000,857.000000,789.000000,4294.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [54]:
binary.describe()

,ContactID,CompanyID,Mapping,Y1,Y2,duration,quantile,webvisit,EmailClickthrough,EmailOpen,...,Industry_산업_제조,isfam_1.0,first_action_1,고객등급_1,고객등급_2,고객등급_3,고객등급_4,고객등급_5,고객등급_6,고객등급_7
count,28697.000000,11594.000000,22017.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,...,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000
mean,19573.246158,1492.058565,0.526593,0.736035,0.055999,168.148122,1.103878,0.764296,0.367739,0.727428,...,0.116911,0.086560,0.163710,0.118096,0.007597,0.024741,0.013660,0.168136,0.020908,0.050876
std,11550.173277,364.523211,0.499304,0.440788,0.229924,175.153949,1.164946,0.424445,0.482198,0.445290,...,0.321320,0.281193,0.370019,0.322727,0.086828,0.155338,0.116077,0.373994,0.143079,0.219749
min,1.000000,899.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8490.000000,1308.000000,0.000000,0.000000,0.000000,0.120000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,20582.000000,1457.000000,1.000000,1.000000,0.000000,105.910000,1.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,29323.000000,1645.000000,1.000000,1.000000,0.000000,325.170000,2.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,38999.000000,3493.000000,1.000000,1.000000,1.000000,614.060000,3.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [55]:
model_path = '../../data/modelling_data/'
continuous.to_csv(model_path + 'model_data_연속형행동.csv',index=False)
binary.to_csv(model_path + 'model_data_이항형행동.csv',index=False)

In [56]:
continuous.shape

(28697, 45)

In [57]:
binary.shape

(28697, 45)